In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(6, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, 800)  # 800 = 400*2 (400 points, 2d)

    def forward(self, x):  # x are the states: [d_obs_x, d_obs_y, d_goal_x, d_goal_y]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=6, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=800, bias=True)
)


In [8]:
import os
import numpy as np
from tqdm import tqdm

data_root_path = '/home/yigit/Documents/projects/irl_sfm/data/demonstrations/sfm/small_env_changing_s_g/used_solid/'

num_traj = 0
x_list, y_list = [], []

for filename in tqdm(os.listdir(data_root_path)):
    if filename.endswith(".npy"):
        filepath = os.path.join(data_root_path, filename)
        config = filename.strip('.npy').split('_')  # format: sx_<>_sy_<>_gx_<>_gy_<>_x_<>_y_<>.npy
        sx, sy = float(config[1]), float(config[3])
        gx, gy = float(config[5]), float(config[7])
        ox, oy = float(config[9]), float(config[11])
        
        x_list.append([sx, sy, gx, gy, ox, oy])
        trajectory = np.load(filepath)
        desired_len = 400
        indices = np.linspace(10, len(trajectory)-1, desired_len, dtype=int)
        y_list.append(trajectory[indices, -2:].tolist())

print(len(x_list))
print(len(y_list))
# print('-----')
# print(x_list[19856])
# print('-----')
# print(y_list[19856])

100%|██████████| 7908/7908 [00:08<00:00, 978.73it/s] 

7908
7908


In [9]:
batch_size = 100
num_instances_train = len(x_list)

x_train = torch.Tensor(x_list)
y_train = torch.Tensor(y_list).view(num_instances_train, desired_len*2)
print(y_train.shape)

torch.Size([7908, 800])


In [10]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

epochs = 2000

for epoch in range(epochs):  # loop over the dataset <epochs> times

    train_ids = torch.randperm(num_instances_train)  # shuffle instances every epoch
    num_iters = int(num_instances_train/batch_size)
    
    running_loss = 0.0
    for i in range(num_iters):
        batch_train_ids = train_ids[i*batch_size:(i+1)*batch_size]
        
        # get the inputs; data is a list of [inputs, labels]
        inputs = x_train[batch_train_ids, :]
        labels = y_train[batch_train_ids, :]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50), end = "\r")
            running_loss = 0.0
            
    if epoch % 10 == 0:
        print()

print('Finished Training')

[1,    50] loss: 14.477
[11,    50] loss: 0.467
[21,    50] loss: 0.224
[31,    50] loss: 0.194
[41,    50] loss: 0.180
[51,    50] loss: 0.185
[61,    50] loss: 0.182
[71,    50] loss: 0.183
[81,    50] loss: 0.180
[91,    50] loss: 0.181
[101,    50] loss: 0.173
[111,    50] loss: 0.174
[121,    50] loss: 0.172
[131,    50] loss: 0.170
[141,    50] loss: 0.168
[151,    50] loss: 0.166
[161,    50] loss: 0.166
[171,    50] loss: 0.165
[181,    50] loss: 0.162
[191,    50] loss: 0.164
[201,    50] loss: 0.160
[211,    50] loss: 0.156
[221,    50] loss: 0.157
[231,    50] loss: 0.153
[241,    50] loss: 0.149
[251,    50] loss: 0.151
[261,    50] loss: 0.147
[271,    50] loss: 0.152
[281,    50] loss: 0.146
[291,    50] loss: 0.146
[301,    50] loss: 0.146
[311,    50] loss: 0.142
[321,    50] loss: 0.137
[331,    50] loss: 0.136
[341,    50] loss: 0.135
[351,    50] loss: 0.131
[361,    50] loss: 0.132
[371,    50] loss: 0.129
[381,    50] loss: 0.130
[391,    50] loss: 0.131
[401,    5

In [11]:
# Saving the trained model
model_root_path = f'/home/yigit/Documents/projects/irl_sfm/python_ws/nn_baseline_for_cnmp/model/nn_models/'
torch.save(net.state_dict(), f'{model_root_path}changing_s_g_model_traj_est_state_dict.pt')